In [36]:
import asf_search as asf
from shapely import Polygon, box
import numpy as np
import pyproj
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.merge
import os
from rasterio.transform import from_origin
import geopandas as gpd
from dem_stitcher import stitch_dem
import zipfile
from urllib.request import urlretrieve
import geopandas as gpd
from shapely import Polygon
import requests
import rioxarray
import matplotlib.pyplot as plt

# Functions

In [59]:
def transform_polygon(src_crs, dst_crs, geometry, always_xy=True):
    src_crs = pyproj.CRS(f"EPSG:{src_crs}")
    dst_crs = pyproj.CRS(f"EPSG:{dst_crs}") 
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=always_xy)
     # Transform the polygon's coordinates
    transformed_exterior = [
        transformer.transform(x, y) for x, y in geometry.exterior.coords
    ]
    # Create a new Shapely polygon with the transformed coordinates
    transformed_polygon = Polygon(transformed_exterior)
    return transformed_polygon

def adjust_scene_poly_at_extreme_lat(bbox, src_crs, ref_crs, delta=0.1):
    """
    Adjust the bounding box around a scene in src_crs (4326) due to warping at high
    Latitudes. For example, the min and max boudning values for an antarctic scene in
    4326 may not actually be the true min and max due to distortions at high latitudes. 

    Parameters:
    - bbox: Tuple of four coordinates (x_min, y_min, x_max, y_max).
    - src_crs: Source EPSG. e.g. 4326
    - ref_crs: reference crs to create the true bbox. i.e. 3031 in southern 
                hemisphere and 3995 in northern (polar stereographic)
    - delta: distance between generation points along the bounding box sides in
            src_crs. e.g. 0.1 degrees in lat/lon 

    Returns:
    - A polygon bounding box expanded to the true min max
    """
    x_min, y_min, x_max, y_max = bbox
    # Generate points along the top side
    top_side = [(x, y_max) for x in list(np.arange(x_min, x_max, delta)) + [x_max]]    
    # Generate points along the right side
    right_side = [(x_max, y) for y in list(np.arange(y_max, y_min, -delta)) + [y_min]]
    # Generate points along the bottom side
    bottom_side = [(x, y_min) for x in list(np.arange(x_max, x_min, -delta)) + [x_min]]
    # Generate points along the left side
    left_side = [(x_min, y) for y in list(np.arange(y_min, y_max, delta)) + [y_max]]
    # Combine all sides' points
    all_points = top_side + right_side + bottom_side + left_side
    # convert to a polygon 
    polygon = Polygon(all_points)
    # convert polygon to desired crs and get bounds in those coordinates
    trans_bounds = transform_polygon(src_crs, ref_crs, polygon).bounds
    trans_poly = Polygon(
        [(trans_bounds[0], trans_bounds[1]), 
         (trans_bounds[2], trans_bounds[1]), 
         (trans_bounds[2], trans_bounds[3]), 
         (trans_bounds[0], trans_bounds[3])]
        )
    corrected_poly = transform_polygon(ref_crs, src_crs, trans_poly)
    return corrected_poly

def reproject_raster(in_path: str, out_path: str, crs: int):
    """Reproject a raster to the desired crs

    Args:
        in_path (str): path to src raster
        out_path (str): save path of reproj raster
        crs (int): crs e.g. 3031

    Returns:
        str: save path of reproj raster
    """
    # reproject raster to project crs
    with rasterio.open(in_path) as src:
        src_crs = src.crs
        transform, width, height = calculate_default_transform(
            src_crs, crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()

        # get crs proj 
        crs = pyproj.CRS(f"EPSG:{crs}")

        kwargs.update({
            'crs': crs,
            'transform': transform,
            'width': width,
            'height': height})

        with rasterio.open(out_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=crs,
                    resampling=Resampling.bilinear)
    return out_path

def find_files(folder, contains):
    paths = []
    for root, dirs, files in os.walk(folder):
        for name in files:
            if contains in name:
                filename = os.path.join(root,name)
                paths.append(filename)
    return paths

def get_REMA_index_file(save_folder):
    rema_index_url = 'https://data.pgc.umn.edu/elev/dem/setsm/REMA/indexes/REMA_Mosaic_Index_latest_gdb.zip'
    filename = 'REMA_Mosaic_Index_latest_gdb.zip'
    # download and store locally
    zip_save_path = os.path.join(save_folder, filename)
    urlretrieve(rema_index_url, zip_save_path)
    #unzip 
    with zipfile.ZipFile(zip_save_path, 'r') as zip_ref:
        zip_ref.extractall(save_folder)
        files=zip_ref.infolist()
        rema_index_file = '/'.join(files[0].filename.split('/')[0:-1])
    rema_index_path = os.path.join(save_folder, rema_index_file)
    os.remove(zip_save_path)
    return rema_index_path

def get_REMA_tiles(s3_url_list, resolution, save_folder):

    valid_res = [2, 10, 32, 100, 500, 1000]
    assert resolution in valid_res, f"resolution must be in {valid_res}"

    # format for request, all metres except 1km
    resolution = f'{resolution}m' if resolution != 1000 else '1km'

    # download individual dems
    dem_paths = []
    for i, s3_file_url in enumerate(s3_url_list):
        # all urls are for 10m, set to other baws on resololution
        s3_file_url = s3_file_url.replace('10m',f'{resolution}')
        # get the raw json url
        json_url = f'https://{s3_file_url.split("external/")[-1]}'
        # Make a GET request to fetch the raw JSON content
        response = requests.get(json_url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse JSON content into a Python dictionary
            data = response.json()
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")
        
        dem_url = json_url.replace('.json', '_dem.tif')
        local_path = os.path.join(save_folder, dem_url.split('amazonaws.com')[1][1:])
        local_folder = '/'.join(local_path.split('/')[0:-1])
        # check if the dem.tif already exists
        dem_path = find_files(local_folder, 'dem.tif')
        if len(dem_path) > 0:
            print(f'{dem_path[0]} already exists, skipping download')
            dem_paths.append(dem_path[0])
            continue
        os.makedirs(local_folder, exist_ok=True)
        print(f'downloading {i+1} of {len(s3_url_list)}: src: {dem_url} dst: {local_path}')
        urlretrieve(dem_url, local_path)
        dem_paths.append(dem_url)

    return dem_paths


def transform_polygon(src_crs, dst_crs, geometry, always_xy=True):
    src_crs = pyproj.CRS(f"EPSG:{src_crs}")
    dst_crs = pyproj.CRS(f"EPSG:{dst_crs}") 
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=always_xy)
     # Transform the polygon's coordinates
    transformed_exterior = [
        transformer.transform(x, y) for x, y in geometry.exterior.coords
    ]
    # Create a new Shapely polygon with the transformed coordinates
    transformed_polygon = Polygon(transformed_exterior)
    return transformed_polygon

def check_s1_bounds_cross_antimeridian(bounds, max_scene_width=20):
    """Check if the s1 bounds cross the antimeridian. We set a max
    scene width of 10. if the scene is greater than this either side
    of the dateline, we assume it crosses the dateline as the
    alternate scenario is a bounds with a very large width.

    e.g. [-178.031982, -71.618958, 178.577438, -68.765755]

    Args:
        bounds (list or tuple): Bounds in 4326
        max_scene_width (int, optional): maxumum width of bounds in 
        lon degrees. Defaults to 20.

    Returns:
        bool: True if crosses the antimeridian
    """

    min_x = -180 + max_scene_width # -160
    max_x = 180 - max_scene_width # 160
    if (bounds[0] < min_x) and (bounds[0] > -180):
        if bounds[2] > max_x and bounds[2] < 180:
            return True
    
def split_am_crossing(scene_polygon, lat_buff=0.2):
    """split the polygon into bounds on the left and
    right of the antimeridian.

    Args:
        scene_polygon (shapely.polygon): polygon of the scene from asf
        lat_buff (float): A lattitude degrees buffer to add/subtract from
                            max and min lattitudes 

    Returns:
    list(tuple) : a list containing two sets of bounds for the left
                    and right of the antimeridian
    """
    max_negative_x = max([point[0] for point in scene_polygon.exterior.coords if point[0] < 0])
    min_positive_x = min([point[0] for point in scene_polygon.exterior.coords if point[0] > 0])
    min_y = min([point[1] for point in scene_polygon.exterior.coords]) - lat_buff
    max_y = max([point[1] for point in scene_polygon.exterior.coords]) + lat_buff
    min_y = -90 if min_y < -90 else min_y
    max_y = 90 if max_y > 90 else max_y
    bounds_left = (-180, min_y, max_negative_x, max_y)
    bounds_right = (min_positive_x, min_y, 180, max_y)
    return [tuple(bounds_left), tuple(bounds_right)]

def reproject_match_tifs(
        tif_1, 
        tif_2, 
        target_crs,
        scene_poly=None,
        save_path_1='',
        save_path_2='',
        convert_dB=False,
        set_nodata=False):
    
    tif_1_f = rioxarray.open_rasterio(tif_1)
    tif_2_f = rioxarray.open_rasterio(tif_2)
    crs_1 = tif_1_f.rio.crs
    crs_2 = tif_2_f.rio.crs
    nodata_1 = tif_1_f.rio.nodata
    nodata_2 = tif_2_f.rio.nodata

    # reproject raster to target crs if not already
    print('reprojecting arrays if not in target crs')
    print(f'target crs: {target_crs}, crs_1: {crs_1}, crs_2: {crs_2}')
    tif_1_reproj = tif_1_f.rio.reproject(f"EPSG:{target_crs}") if str(tif_1_f.rio.crs) != str(target_crs) else tif_1_f
    tif_2_reproj = tif_2_f.rio.reproject(f"EPSG:{target_crs}") if str(tif_2_f.rio.crs) != str(target_crs) else tif_2_f
    
    del tif_1_f, tif_2_f
    print('Shape of arrays after reprojection to target crs')
    print(tif_1_reproj.shape, tif_2_reproj.shape)
    # clip by the scene geometry
    if scene_poly is not None:
        print('clip arrays by the scene bounds')
        tif_1_clipped = tif_1_reproj.rio.clip([scene_poly], pyproj.CRS.from_epsg(4326))
        tif_2_clipped = tif_2_reproj.rio.clip([scene_poly], pyproj.CRS.from_epsg(4326))
        print('Shape of arrays after being clipped by scene bounds')
        print(tif_1_clipped.shape, tif_2_clipped.shape)
    else:
        tif_1_clipped = tif_1_reproj.copy()
        tif_2_clipped = tif_2_reproj.copy()

    del tif_1_reproj, tif_2_reproj
    # match the shape and resolution of the two tifs as they may be slighly off
    # match tif 2 to tif 1 if tif 1 did not require reprojection originally
    print('matching shape and resolutions')
    if str(crs_1) == str(target_crs):
        print('reprojecting tif 2 to tif 1')
        tif_2_matched = tif_2_clipped.rio.reproject_match(tif_1_clipped)
        tif_1_matched = tif_1_clipped.copy()
    else:
        print('reprojecting tif 1 to tif 2')
        tif_1_matched = tif_1_clipped.rio.reproject_match(tif_2_clipped)
        tif_2_matched = tif_2_clipped.copy()
    del tif_1_clipped, tif_2_clipped
    print('Shape of arrays after matching tifs')
    print(tif_1_matched.shape, tif_2_matched.shape)

    if set_nodata:
        print(f'setting nodata values to {set_nodata}')
        tif_1_matched = tif_1_matched.where(nodata_1,set_nodata)
        tif_2_matched = tif_2_matched.where(nodata_2,set_nodata)
        tif_1_matched.rio.write_nodata(set_nodata, encoded=True, inplace=True)
        tif_2_matched.rio.write_nodata(set_nodata, encoded=True, inplace=True)

    if convert_dB:
        print('converting from linear to db')
        print(f'converting tif 1...')
        tif_1_matched = 10*np.log10(tif_1_matched)
        print(f'converting tif 2...')
        tif_2_matched = 10*np.log10(tif_2_matched)

    if save_path_1:
        print(f'Saving to: {save_path_1}')
        tif_1_matched.rio.to_raster(save_path_1)
    if save_path_2:
        print(f'Saving to: {save_path_2}')    
        tif_2_matched.rio.to_raster(save_path_2)

    return tif_1_matched, tif_2_matched

def plot_difference_maps(
    arr_1, 
    arr_2, 
    titles=['arr_1','arr_2','diff (arr_1 - arr_2)'],
    scales = [[-40,10],[-40,10],[-1,1]],
    ylabels=['decibels (dB)','decibels (dB)','decibels difference (dB)'],
    save_path='',
    exclude_nodata=True,
    show=False):
    
    if isinstance(arr_1, str):
        with rasterio.open(arr_1) as src:
                arr_1 = src.read(1)
    if isinstance(arr_2, str):
        with rasterio.open(arr_2) as src:
                arr_2 = src.read(1)
    
    # ensure shapes are the same, required for difference
    assert arr_1.shape == arr_2.shape, "tifs are two different shapes, must be identical for difference calculation"
    
    # only compare where we have data in both tifs
    if exclude_nodata:
        arr_2[(~np.isfinite(arr_1))] = np.nan
        arr_1[(~np.isfinite(arr_2))] = np.nan

    diff = arr_1 - arr_2
    arrs = [arr_1, arr_2, diff]
    stats_arr = np.array(diff)[np.array((np.isfinite(diff)))]
    print('Difference Stats')
    print(f'min: {stats_arr.min()}', 
        f'max: {stats_arr.max()}',
        f'mean: {stats_arr.mean()}',
        f'median: {np.percentile(stats_arr, 50)}',
        f'5th percentile: {np.percentile(stats_arr, 5)}',
        f'90th percentile: {np.percentile(stats_arr, 95)}',
        )

    cmaps = ['binary_r','binary_r','bwr']

    f, ax = plt.subplots(nrows=4, ncols=1, figsize=(10,40))
    for i,arr in enumerate(arrs):
        im = ax[i].imshow(arr, 
                vmin = scales[i][0], 
                vmax = scales[i][1],
                cmap = cmaps[i])
        ax[i].set_title(titles[i])
        f.colorbar(im, ax=ax[i], label=ylabels[i])
        
    # plot the histogram
    colors = ['red','blue']
    for i in [0,1]:
        # only get real values 
        hist_data = np.array(arrs[i])[
                (np.isfinite(np.array(arrs[i])))
                ]
        u, std = np.mean(hist_data), np.std(hist_data)
        ax[3].hist(hist_data, 
                density=True,
                bins=60, 
                alpha=0.5, 
                label=f'{titles[i]}; u={u:.3f}, std={std:.3f}', 
                color=colors[i],
                histtype='step')
        ax[3].set_title('Pixel distribution')

    plt.legend(loc='best')
    if save_path:
        plt.savefig(save_path)

    if show:
        plt.show()

def reassign_nodata(tif_path: str, nodata, outpath='', dtype='', reassign_values_to_nodata=[]):
    """re assigns the nodata value in the tif

    Args:
        tif_path (str): path to tif
        nodata : value for nodata 
        outpath : save to path, else overwrite if empty
        dtype : convert to dtype, else keep same if empty
    """
    tif = rioxarray.open_rasterio(tif_path)
    dtype_orig = tif.dtype
    if dtype:
        print(f'changing {tif_path} dtype from {dtype_orig} to {dtype}')
        tif = tif.astype(dtype)
    nodata_orig = tif.rio.nodata
    print(f'changing {tif_path} nodata from {nodata_orig} to {nodata}')
    tif.where(nodata_orig,nodata)
    if reassign_values_to_nodata:
        for value in reassign_values_to_nodata:
            tif = tif.where(tif != value, other=nodata)
    tif.rio.write_nodata(nodata, inplace=True)
    if outpath:
        tif.rio.to_raster(outpath)
        return outpath
    else:
        tif.rio.to_raster(tif_path)
        return tif_path

Search for sentinel 1 scenes to get some geographical bounds

In [ ]:
prod = 'SLC'
mode = 'IW'

scene_list = [
    # antimeridian scene
    #'S1A_IW_SLC__1SDV_20191028T131928_20191028T131947_029659_0360CA_A1A0',

    # high latittude scene - Ross Island 
    'S1A_IW_SLC__1SSH_20211220T124745_20211220T124815_041092_04E1C2_0475'
]

# search the scene list with the specified product type
results = asf.granule_search(scene_list, asf.ASFSearchOptions(processingLevel=prod))
print(f'number of results found : {len(results)}')

Search for bounds

In [4]:
# prod= 'GRD'
# mode = 'EW'
# wkt = "POINT (-24.984 -73.620)" 
# start='2024-02-29T11:59:59Z'
# end='2024-03-31T11:59:59Z'

# # search the scene list with the specified product type
# results = asf.search(
#     platform=[asf.PLATFORM.SENTINEL1],
#     intersectsWith= wkt,
#     maxResults=1000,
#     beamMode=mode,
#     #processingLevel=prod,
#     start=start,
#     end=end
#     )
# print(f'number of results found : {len(results)}')
# for r in results:
#     print(r.properties['sceneName'])

In [ ]:
geom = results[0].geometry
scene_polygon = Polygon(geom['coordinates'][0])
bounds = scene_polygon.bounds
bounds

In [ ]:
scene_polygon

In [7]:
scene_polygon_buf = scene_polygon.buffer(0.1)
bounds_buf = scene_polygon_buf.bounds

## Compernicus 30m

In [ ]:
antimeridian_crossing = check_s1_bounds_cross_antimeridian(bounds, max_scene_width=8)
if antimeridian_crossing:
    trg_crs = 3031
    print('crossing AM')
    # split into bounds either side of the AM
    bounds_left, bounds_right = split_am_crossing(scene_polygon, lat_buff=0.2)
    # get left dem
    bounds_left = adjust_scene_poly_at_extreme_lat(bounds_left, 4326, 3031).bounds
    dem_data, dem_meta = stitch_dem(bounds_left,
        dem_name='glo_30',
        dst_ellipsoidal_height=True,
        dst_area_or_point='Point',
        merge_nodata_value=0,
        fill_to_bounds=True,
    )
    with rasterio.open('tmp_left_dem.tif', 'w', **dem_meta) as ds:
        ds.write(dem_data,1)
        ds.update_tags(AREA_OR_POINT='Point')
    # reproject left dem to target crs
    #   - warning dem is at AM, crs to handle this is required
    #   - raise error if 4326?
    reproject_raster('tmp_left_dem.tif',f'tmp_left_dem_{trg_crs}.tif',trg_crs)
    # get right dem
    bounds_right = adjust_scene_poly_at_extreme_lat(bounds_right, 4326, 3031).bounds
    dem_data, dem_meta = stitch_dem(bounds_right,
        dem_name='glo_30',
        dst_ellipsoidal_height=True,
        dst_area_or_point='Point',
        merge_nodata_value=0,
        fill_to_bounds=True,
    )
    # check target crs
    with rasterio.open('tmp_right_dem.tif', 'w', **dem_meta) as ds:
        ds.write(dem_data,1)
        ds.update_tags(AREA_OR_POINT='Point')
    # reproject right dem to target crs
    reproject_raster('tmp_right_dem.tif',f'tmp_right_dem_{trg_crs}.tif',trg_crs)
    # merge dems
    rasterio.merge.merge(
        sources=[f'tmp_left_dem_{trg_crs}.tif',f'tmp_right_dem_{trg_crs}.tif'],
        dst_path='dem.tif')
    # merge dems

else:
    print('not crossing AM')
    new_poly = adjust_scene_poly_at_extreme_lat(bounds, 4326, 3031, delta=0.01)
    new_bounds = new_poly.bounds
    dem_data, dem_meta = stitch_dem(new_bounds,
        dem_name='glo_30',
        dst_ellipsoidal_height=True,
        dst_area_or_point='Point',
        merge_nodata_value=0,
        fill_to_bounds=True,
    )
    with rasterio.open('cop30m_dem.tif', 'w', **dem_meta) as ds:
        ds.write(dem_data,1)
        ds.update_tags(AREA_OR_POINT='Point')

# REMA DEM

In [8]:
# convert to 3031
scene_poly_3031 = transform_polygon(4326, 3031, scene_polygon)
# buffer by 10km each size
buf = 10_000
scene_poly_3031_buf = scene_poly_3031.buffer(buf)

In [9]:
resolution = 32 # must be one of [2, 10, 32, 100, 500, 1000]
dem_filename = f'REMA{resolution}_dem.tif'

In [ ]:
os.makedirs('data', exist_ok=True)
print('Downloading REMA index file')
rema_index_path = get_REMA_index_file(save_folder='data')

In [ ]:
# load into gpdf
rema_index_df = gpd.read_file(rema_index_path)
# find the intersecting tiles
intersecting_rema_files = rema_index_df[
    rema_index_df.geometry.intersects(scene_poly_3031_buf)]
s3_url_list = intersecting_rema_files['s3url'].to_list()
print(f'{len(s3_url_list)} intersecting tiles found')
dem_paths = get_REMA_tiles(s3_url_list[0:], resolution, save_folder='data')

In [ ]:
# combine DEMS
# for some reason this may hand on first run, you may need to restart and run again
print('combining DEMS')
rasterio.merge.merge(
        datasets=dem_paths,
        dst_path=dem_filename
        )


# reporject to desired crs
# if str(crs) != '3031':
#     reproj_dem_path = os.path.join('data',dem_filename)
#     print(f'reprojecting DEM to {crs}')
#     reproject_raster(merge_dem_path, reproj_dem_path, crs)
#     os.remove(merge_dem_path)
#     return reproj_dem_path


# align the tifs in shape and crs

In [ ]:
# note - need to sort out issues with nodata etc
reproject_match_tifs(
        'REMA32_dem.tif',
        'cop30m_dem.tif', 
        target_crs=3031,
        scene_poly=None,
        save_path_1='REMA32_dem_match.tif',
        save_path_2='cop30m_dem_match.tif',
        convert_dB=False,
        set_nodata=np.nan)

In [ ]:
reassign_nodata(tif_path='REMA32_dem_match.tif', nodata=0, reassign_values_to_nodata=[-9999])
reassign_nodata(tif_path='cop30m_dem_match.tif', nodata=0, reassign_values_to_nodata=[-9999])

# take the difference between tifs

In [57]:
diff = rioxarray.open_rasterio('REMA32_dem_match.tif') - rioxarray.open_rasterio('cop30m_dem_match.tif')
diff.rio.to_raster('difference.tif')

In [ ]:
plot_difference_maps(
    arr_1='REMA32_dem_match.tif', 
    arr_2='cop30m_dem_match.tif', 
    titles=['REMA32_dem','cop30m_dem','diff (REMA32_dem - cop30m_dem)'],
    scales = [[-50,2000],[-50,2000],[0,50]],
    ylabels=['REMA32_dem (m)','cop30m_dem (m)','difference (m)'],
    save_path='',
    exclude_nodata=True,
    show=True
    )